<a href="https://colab.research.google.com/github/ejdautel/AlgaeNation/blob/master/AlgaeNation_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###Compiled

#Import Libraries
import numpy as np
from scipy.optimize import root

'________________________________________________________________________________________________________________________________________________________'
#CONSTANTS

##Optimal Bench Scale Flowrates 
air_inlet=30.00 #SCFH
gas_inlet_conc = 0.15 #inlet CO2 gas Concentration (CO2 gas flow/Total gas flow)
gas_outlet_conc = 0.092 #outlet CO2 gas Concnetration
CO2_inlet=(air_inlet*gas_inlet_conc)/(1-gas_inlet_conc) #SCFH
CO2_outlet=(air_inlet*0.092)/(1-0.092) #SCFH
solvFR=20.00 #GPH
LiqVolRat= 76.00 #L/m^3

##Material Properties
CO2_den=	1.98	#g/L Density of CO2
CO2_mass=	44.01	#g/mol Mass of CO2
air_den=	1.23	#g/L Density of Air
air_mass=	28.96	#g/mol Mass of Air
CO2_den_air=	1.30	#g/L Density of 15% CO2 in Air
water_den=	1000.00	#g/L Density of Water
water_mass=	18.00	#g/mol Mass of Water
K2CO3_den=	2430.00	#g/L Density of K2CO3
K2CO3_mass=	138.21	#g/mol Mass of K2CO3
K2CO3_mix_den=	1096.82	#g/L Density of 15% mixture K2CO3
K2CO3_cost_lb= 1.1635 #$/lb Cost of K2CO3 per lb

##Coversions
ft3_L = 28.32 #L/ft3
hr_s = 3600.00 #s/hr
gal_L = 3.79 #L/gal
gph_Lps = 951.02 #gph/Lps
ft3_m3 = 35.31 #ft3/m3
m3_L = 1000.00 #L/m3
lb_g = 453.592 #g/lb
ton_lb = 2000 #lb/lb-ton
lb_kg = 2.2 #lb/kg

##Bench Scale Column Specs
h_b = 0.50 #m (Column Packing Height)
sa_b = 0.17 #m^2 (Packing Surface Area)
r_b = 0.05 #m (Column Radius)
A_b = (np.pi)*(r_b**2) #m^2 (Column Cross-Sectional Area)
V_b = h_b*A_b


##Maximum Flue Gas Demand (GD)
demand_per_site = 19.70 #ft3/min
demand_per_1_PA = 0.13 #ft3/min (1 m^2 Pond)
demand_per_3_PA = 0.44 #ft3/min (3.5 m^2 Pond)
demand_per_43_PA = 5.38 #ft3/min (43 m^2 Pond)
GD_per_site = demand_per_site/ft3_m3/60 #ft3/min
GD_per_1_PA = demand_per_1_PA/ft3_m3/60 #ft3/min (1 m^2 Pond)
GD_per_3_PA = demand_per_3_PA/ft3_m3/60 #ft3/min (3.5 m^2 Pond)
GD_per_43_PA = demand_per_43_PA/ft3_m3/60 #ft3/min (43 m^2 Pond)

##Calculated Solvent Demand (SD) using L/V Ratio		
SD_per_site = GD_per_site*LiqVolRat	#L/s Per site
SD_per_1_PA = GD_per_1_PA*LiqVolRat	#L/s Per 1 m^2 pond area
SD_per_3_PA = GD_per_3_PA*LiqVolRat	#L/s Per 3.5 m^2 pond area
SD_per_43_PA = GD_per_43_PA*LiqVolRat	#L/s Per 43 m^2 pond area

'________________________________________________________________________________________________________________________________________________________'
#Calculated Bench Scale Input Constants
##Molar Flow Rates

Bench_CO2_inlet = CO2_inlet* ft3_L/ hr_s* CO2_den/CO2_mass #mol/s 
                  #5.3[SCFH] * 23.32 [L/ft3] / 3600 [s/hr] * 1.98 [g/L] / 44.01 [g/mol] 
Bench_CO2_outlet = CO2_outlet* ft3_L / hr_s * CO2_den/ CO2_mass #mol/s
                  #3.04[SCFH] * 23.32 [L/ft3] / 3600 [s/hr] * 1.98 [g/L] / 44.01 [g/mol]
Bench_CO2_absorbed = Bench_CO2_inlet - Bench_CO2_outlet
Bench_air_flowrate = air_inlet*ft3_L/ hr_s*air_den/ air_mass #mol/s 
                  #30[SCFH] * 23.32 [L/ft3] / 3600 [s/hr] * 1.23 [g/L] / 28.96 [g/mol] 
Bench_vapor_flowrate = Bench_CO2_inlet + Bench_air_flowrate #mol/s
Bench_K2CO3_usage = solvFR/ gph_Lps * K2CO3_mix_den* gas_inlet_conc / K2CO3_mass #mol/s 
                  #20 [GPH] / 951.02 [GPH/LPS] * 1096.82[g/L] * 0.15 / 138.21 [g/mol]
Bench_water_usage = solvFR/ gph_Lps * K2CO3_mix_den* (1-gas_inlet_conc) / water_mass #mol/s 
                  #20 [GPH] / 951.02 [GPH/LPS] * 1096.82[g/L] * 0.85 / 18 [g/mol]
Bench_total_solv_flow = Bench_K2CO3_usage+ Bench_water_usage #mol/s

##Molar Composition
x_in = 0
x_out = Bench_CO2_absorbed/( Bench_CO2_absorbed + Bench_total_solv_flow)
y_in= Bench_CO2_inlet/ Bench_vapor_flowrate
y_out= Bench_CO2_outlet/ Bench_vapor_flowrate

'________________________________________________________________________________________________________________________________________________________'
#Dynamic Industrial Molar Flow Rates based on User Input of Pond Size
def Dynamic_Flows():
  global vapor_flowrate,total_solv_flow,total_K2CO3_usage,total_water_usage
  GD,SD = Pond_Area()
  CO2_inlet =((GD* gas_inlet_conc )/(1- gas_inlet_conc ))* m3_L * CO2_den / CO2_mass #mol/s 
  CO2_outlet =((GD* gas_outlet_conc )/(1- gas_outlet_conc ))* m3_L * CO2_den / CO2_mass #mol/s
  CO2_absorbed = CO2_inlet - CO2_outlet
  air_flowrate =((GD* (1-gas_inlet_conc) )/(gas_inlet_conc))* m3_L * air_den / air_mass #mol/s 
  vapor_flowrate = CO2_inlet + air_flowrate #mol/s
  K2CO3_usage = SD*K2CO3_mix_den*gas_inlet_conc/K2CO3_mass #mol/s 
  #solvFR/ gph_Lps * K2CO3_mix_den* gas_inlet_conc / K2CO3_mass 
  water_usage = SD* K2CO3_mix_den*(1-gas_inlet_conc)/ water_mass #mol/s 
  #solvFR/ gph_Lps * K2CO3_mix_den* (1gas_inlet_conc) / water_mass 
  total_solv_flow = K2CO3_usage+ water_usage #mol/s

  ####K2CO3 and Water Usage
  total_K2CO3_usage = K2CO3_usage* K2CO3_mass/m3_L* hr_s *24 #kg/day
  total_water_usage = water_usage*water_mass / water_den / gal_L * hr_s * 24 #kg/day
  
'________________________________________________________________________________________________________________________________________________________'
#Scale-up Dimensions based on User Inputs for Height and Pond Size
def find_r(given_h):
  #Bench scale Henry Constant from Phase 2 Experiments
  K = 0.325 #This will need to change in the future based on the solvent concentration and Flowrate Ratio
  
  Dynamic_Flows()
  
  #Bench Scale design equation to find the mass transfer coefficient
  Batch_LS_DE = np.log(((K*x_in)-y_out)/((K*x_out)-y_in))
  Batch_Mid_DE = (((K*Bench_vapor_flowrate )/Bench_total_solv_flow)-1)*((sa_b*A_b*h_b)/Bench_vapor_flowrate )
  Batch_RS_DE = (1-(Bench_total_solv_flow/(K*Bench_vapor_flowrate)))*((sa_b*A_b*h_b)/Bench_total_solv_flow)

  Kx=Batch_RS_DE/Batch_Mid_DE
  Ky=Batch_LS_DE/Batch_Mid_DE

  #Design Equation to find the mass transfer coefficient
  LS_DE = Batch_LS_DE
  Mid_DE = (((K*vapor_flowrate)/total_solv_flow)-1)*((Ky)/vapor_flowrate)

  #Calculating the product of Surface Area, Cross Sectional Area, and Height for Industrial Column
  sa_A_h = LS_DE/Mid_DE

  #Desired height
  given_h 

  #Initial radius guess
  r = 0.2 

  #Define function to solve for the radius and store the results
  def solve(r):
    sa_l = (2*np.pi*r*given_h)+(2*np.pi*(r**2)) #m^2
    A_l = np.pi*r**2 #m^2
    h_l = given_h #m
    Calc_sa_A_h = sa_l*A_l*h_l
    set_produce_to_zero = Calc_sa_A_h-sa_A_h
    return abs(set_produce_to_zero)

  Solver_results = root(solve,r)

  #Store the solved radius value
  Per_site_r = Solver_results.x 

  return np.round(Per_site_r,4)

In [ ]:
def Column_Cost(plot=0):
  #Create a list of CO2 cost based on height range
  # the radius found using find_r keeps giving me an array value. This makes sa_l an array and therefore Cost_CO2_Per_site is an array i cant figure out how to convert the found r to a scalar
  
  global h_results,r_results,sa_results,vl_results,column_cost_results
  
  bench_cost=5302.00
  packing_cost=500 #$/m^3 of packing
  factor=0.6
  
  store = np.zeros((24,5))
  
  for height in range (1,25):
    r = find_r(height)[0]
    sa_l = (2*np.pi*(r)*height)+(2*np.pi*((r)**2)) #m^2
    A_l = np.pi*(r)**2 #m^2
    V_l = height*A_l
    Cost_coulmn_per_site=((V_l/V_b)**factor)*bench_cost + 500*V_l

    column_names = ["height","r", "Packing Surface Area","Column Volume","Cost of Column Per Site"]
    values = [height,r,sa_l,V_l,Cost_coulmn_per_site]
    store[height-1] = np.round([height,r,sa_l,V_l,Cost_coulmn_per_site],2)

  #Stop showing results as scientific notation
  np.set_printoptions(suppress=True)
  #print the radius results
  #print(column_names[1],store[:,1])

  #generate variables from the results for later use
  h_results = store[:,0]
  r_results = store[:,1]
  sa_results = store[:,2]
  vl_results = store[:,3]
  column_cost_results = store[:,4]

  #If desired, output the Column Cost vs Height plot
  if plot == 1:
    Column_Cost()
    plt.plot(h_results,column_cost_results)
    plt.title(np.char.add('Cost of Column vs Height: ',dashboard_one.pond_size.value))
    plt.xlabel('Height')
    plt.ylabel('Cost of Column')
    plt.show()

In [ ]:
def CO2_Process_Cost():
  
  Column_Cost()

  #Cost of CO2 processed per Ton for Per Site
  CO2_processed_mol_per_s = total_solv_flow*x_out #mol/s
  CO2_processed_lbton_per_yr = CO2_processed_mol_per_s*CO2_mass/lb_g/ton_lb*hr_s*24*365 #lb-ton/yr
  #K2CO3 Utilized (No Recycle)
  K2CO3_cost_per_year = total_K2CO3_usage*lb_kg*365*K2CO3_cost_lb
  Per_site_total_water_usage

  Cost_per_ton_CO2_processed = (column_cost_results+K2CO3_cost_per_year)/CO2_processed_lbton_per_yr

  plt.plot(h_results,Cost_per_ton_CO2_processed)
  plt.title('Per Site')
  plt.xlabel('Height')
  plt.ylabel('Cost per lb-ton CO2 Processed')
  plt.show()

In [ ]:
#Breakkkk

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets
import ipywidgets as widgets
import IPython
from IPython.display import display,clear_output

import webbrowser
from IPython.display import YouTubeVideo

from fastai.vision import *
from fastai.widgets import *
from fastai.callbacks import*

In [ ]:
def version():
    import fastai
    import os
    import tensorflow as tf
    import torch

    print ('>> Scale_up_Analysis_UI Last Update: 4/23/2020 \n\n>> Program info \n')

    button = widgets.Button(description='Program Info')
    display(button)

    out = widgets.Output()
    display(out)

    def on_button_clicked_info(b):
        with out:
            clear_output()
            print(f'Fastai Version: {fastai.__version__}')
            print(f'Python version: {sys.version}')

    button.on_click(on_button_clicked_info)

def dashboard_one():
    style = {'description_width': 'initial'}

    print('>> Currently only works with files FROM_FOLDERS' '\n')

    dashboard_one.gas_conc = widgets.ToggleButtons(
        options=['15%'],
        description='Inlet CO2 Gas Concentration:',
        disabled=False,
        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['15% Inlet Gas Concentration'],
        style=style
    )
    
    dashboard_one.solvent_conc = widgets.ToggleButtons(
        options=['15%','20%','25%'],
        description='Potassium Carbonate Solvent Concentration:',
        disabled=False,
        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['15%','20%','25%'],
        style=style
    )

    dashboard_one.flow_ratio = widgets.ToggleButtons(
        options=['56','76','114'],
        description='Liquid-Gas Flow Rate Ratio (L/m^3):',
        disabled=False,
        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['56 L/m^3','76 L/m^3','114 L/m^3'],
        style=style
    )

    dashboard_one.pond_size = widgets.ToggleButtons(
        options=['Per Site','1 m^2','3.5 m^2','43 m^2'],
        description='Pond Size:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=[],
    )

    button = widgets.Button(description='Radius Result')
    display(button)

    out = widgets.Output()
    display(out)

    def on_button_clicked_info(b):
        with out:
            clear_output()
            print("Chosen Radius is: ", dashboard_one.h.value, " meters.")
            determined_r = find_r(dashboard_one.h.value)
            print("The Scaled-up Radius should be: ", determined_r[0], " meters.")

    button.on_click(on_button_clicked_info)



    button2 = widgets.Button(description='Column Cost')
    display(button2)

    out2 = widgets.Output()
    display(out2)

    def on_button_clicked_column(b):
        with out:
            clear_output()
            Column_Cost(1)

    button2.on_click(on_button_clicked_column)



    button3 = widgets.Button(description='CO2 Processing Cost')
    display(button3)

    out3 = widgets.Output()
    display(out3)

    def on_button_clicked_processing(b):
        with out:
            clear_output()
            CO2_Process_Cost()

    button3.on_click(on_button_clicked_processing)



    layout = widgets.Layout(width='auto', height='40px') #set width and height


    layout = {'width':'90%', 'height': '50px', 'border': 'solid', 'fontcolor':'lightgreen'}
    layout_two = {'width':'100%', 'height': '200px', 'border': 'solid', 'fontcolor':'lightgreen'}
    style_green = {'handle_color': 'green', 'readout_color': 'red', 'slider_color': 'blue'}
    style_blue = {'handle_color': 'blue', 'readout_color': 'red', 'slider_color': 'blue'}
    dashboard_one.h=widgets.FloatSlider(min=0,max=64,step=1,value=32, continuous_update=False, layout=layout, style=style_green, description="Height")

    display(dashboard_one.gas_conc, dashboard_one.solvent_conc, dashboard_one.flow_ratio, dashboard_one.pond_size,dashboard_one.h)


In [ ]:
def Pond_Area():
    if dashboard_one.pond_size.value == 'Per Site':
        Pond_Area.GD = GD_per_site
        Pond_Area.SD = SD_per_site
    elif dashboard_one.pond_size.value == '1 m^2':
        Pond_Area.GD = GD_per_1_PA
        Pond_Area.SD = SD_per_1_PA
    elif dashboard_one.pond_size.value == '3.5 m^2':
        Pond_Area.GD = GD_per_3_PA
        Pond_Area.SD = SD_per_3_PA
    elif dashboard_one.pond_size.value == '43 m^2':
        Pond_Area.GD = GD_per_43_PA
        Pond_Area.SD = SD_per_43_PA

    return Pond_Area.GD,Pond_Area.SD


In [ ]:
def colab_ui():
    from google.colab import widgets
    from google.colab import output

    t = widgets.TabBar(['Info','Process Parameters'])

    t.clear_tab(0)

    with t.output_to(0, select=False):
         version()

    with t.output_to(1):
        t.clear_tab()
        dashboard_one()

In [ ]:
colab_ui()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

ToggleButtons(button_style='info', description='Potassium Carbonate Solvent Concentration:', options=('15%', '…

ToggleButtons(button_style='info', description='Liquid-Gas Flow Rate Ratio (L/m^3):', options=('56', '76', '11…

ToggleButtons(description='Pond Size:', options=('Per Site', '1 m^2', '3.5 m^2', '43 m^2'), value='Per Site')

FloatSlider(value=32.0, continuous_update=False, description='Height', layout=Layout(border='solid', height='5…

<IPython.core.display.Javascript object>